In [21]:
import sqlite3
import requests
import base64
import json
import urllib
import numpy as np

In [4]:
def get_client_credentials_token():
    client_id = "8f579d1c406a4330aa7d14b7bb2806ef"
    client_secret = "3e696c1315764554bec59f8dc611bc1c"

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Authorization": f"Basic {base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode()}"
    }

    data = {
        "grant_type": "client_credentials"
    }

    url = "https://accounts.spotify.com/api/token"
    response = requests.post(url, headers=headers, data=data)
    return response.json()["access_token"]

In [15]:
def generate_recommendations():
    # Get the access token
    token = get_client_credentials_token()

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }

    # Get the past 10 songs played from the database
    past_songs = get_past_songs(40)
    if not past_songs:
        return None

    # Get the Spotify IDs of the past songs
    track_ids = []
    for song in past_songs:
        track_id = get_track_id(song[0], token=token)
        if track_id:
            track_ids.append(track_id)

    if not track_ids:
        return None

    # Get the recommendations from Spotify
    url = f"https://api.spotify.com/v1/recommendations?seed_tracks={','.join(track_ids)}"
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    recommendations = data.get("tracks", [])

    return recommendations

In [7]:
def get_past_songs(limit):
    conn = sqlite3.connect('music.db')
    cursor = conn.cursor()
    cursor.execute("SELECT title, url FROM music ORDER BY id DESC LIMIT ?", (limit,))
    past_songs = cursor.fetchall()
    conn.close()
    return past_songs

In [9]:
def get_track_id(title, token=None):
    # Search for the song on Spotify and return its ID
    if token == None:
        token = get_client_credentials_token()

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }

    query = f"{title}"
    query = urllib.parse.quote(query)
    url = f"https://api.spotify.com/v1/search?q={query}&type=track"
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    try:
        track_id = data.get("tracks", {}).get("items", [{}])[0].get("id")
    except FileNotFoundError:
        track_id = None
    return track_id

In [48]:
def extract_features(songs, token=None):
    if token == None:
        token = get_client_credentials_token()

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }

    features = np.array([])
    keys = ['danceability', 'energy', 'valence', 'key', 'loudness', 'mode', 
            'speechiness', 'acousticness', 'instrumentalness', 'liveness', 
            'valence', 'tempo']
    for i, song in enumerate(songs):
        track_id = get_track_id(song[0], token=token)
        if track_id:
            url = f"https://api.spotify.com/v1/audio-features/{track_id}"
            response = requests.get(url, headers=headers)
            data = json.loads(response.text)
            if features.size == 0:
                features = np.array([data[key] for key in keys])
            else:
                features = np.vstack((features, np.array([data[key] for key in keys])))
    return features

In [62]:
songs = get_past_songs(125)
dataset = extract_features(songs)

In [63]:
from sklearn.preprocessing import StandardScaler

In [64]:
scaler = StandardScaler()
scaled_dataset = scaler.fit_transform(dataset)

In [65]:
dataset.shape

(106, 12)